In [75]:
#This file will require pandas and Numpy.

In [76]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [77]:
# download data files for 160903, 160910 and 160917

In [78]:
weeks=[160903,160910, 160917]
#make into text files


def get_data(datelist):
    pdlist=[]
    datelist=[str(date) for date in datelist]
    for date in datelist:
        url="http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt".format(date)
        pdlist.append(pd.read_csv(url))
    return pd.concat(pdlist)

df= get_data(weeks)
    

In [79]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041
1,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,04:00:00,REGULAR,5799463,1966044
2,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,08:00:00,REGULAR,5799492,1966079
3,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,12:00:00,REGULAR,5799610,1966155
4,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,16:00:00,REGULAR,5799833,1966214


In [80]:
#confirm get_data worked correctly

In [81]:
print(df['DATE'].value_counts())

09/06/2016    28826
09/16/2016    28160
09/08/2016    28137
09/01/2016    28116
09/12/2016    28074
09/07/2016    27977
09/09/2016    27956
08/29/2016    27828
08/30/2016    27775
09/02/2016    27478
09/14/2016    27359
09/03/2016    27353
08/31/2016    27352
09/13/2016    27343
09/11/2016    27334
09/05/2016    27325
09/10/2016    27325
09/04/2016    27304
08/28/2016    27300
08/27/2016    27290
09/15/2016    27283
Name: DATE, dtype: int64


In [82]:
# Next we need to turn this into a time series. There is a column date and time that we want to convert to datetime objects. 
#convert using either pd.to_datetime, or python datetime library to make a new column of datetime type.
#added time delta import statement above in inports

In [83]:
df['datetime']=pd.to_datetime(df['DATE']+" "+df['TIME'])
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041,2016-08-27 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,04:00:00,REGULAR,5799463,1966044,2016-08-27 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,08:00:00,REGULAR,5799492,1966079,2016-08-27 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,12:00:00,REGULAR,5799610,1966155,2016-08-27 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,16:00:00,REGULAR,5799833,1966214,2016-08-27 16:00:00


In [84]:
#test that it is of the datetime data type
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 580895 entries, 0 to 192877
Data columns (total 12 columns):
 #   Column                                                                Non-Null Count   Dtype         
---  ------                                                                --------------   -----         
 0   C/A                                                                   580895 non-null  object        
 1   UNIT                                                                  580895 non-null  object        
 2   SCP                                                                   580895 non-null  object        
 3   STATION                                                               580895 non-null  object        
 4   LINENAME                                                              580895 non-null  object        
 5   DIVISION                                                              580895 non-null  object        
 6   DATE                        

In [85]:
# The combination of C/A, Unit, SCP and station represent a unique turnstile.
#obtain the max extries for each day, for each unique turnstile

In [86]:
turndata=df.groupby([ 'STATION', 'C/A', 'UNIT', 'SCP', 'DATE']).agg({'ENTRIES': ['max']})
turndata.head()

ENTRIES
                                            max
STATION C/A  UNIT SCP      DATE                
1 AV    H007 R248 00-00-00 08/27/2016  11450095
                           08/28/2016  11451910
                           08/29/2016  11455137
                           08/30/2016  11458724
                           08/31/2016  11462054

# Exercise  3b:

In [87]:
# now we want to find the maximum entries per day. This is based on the values from the day before.
df['TIME'].value_counts()

08:00:00    50661
00:00:00    50660
04:00:00    50655
20:00:00    50616
12:00:00    50610
            ...  
11:36:25        1
15:48:25        1
20:00:25        1
02:34:02        1
15:48:46        1
Name: TIME, Length: 20813, dtype: int64

In [88]:
#There are some outlier times, but the true data set is every 4 hours, 0,4,8,12,16 and 20. 
#So that means the the min for day X is at 0:00:00, and max for day x is at 0:00:00 at day x+1

In [89]:
mask=(df.TIME=='00:00:00')
max_and_min=df[mask]
max_and_min.rename(columns={'ENTRIES':'ENTRIES_START'}, inplace=True)
max_and_min.head()

/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_43109/1351586889.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_and_min.rename(columns={'ENTRIES':'ENTRIES_START'}, inplace=True)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES_START,EXITS,datetime
0,A002,R051,02-00-00,59 ST,NQR456,BMT,08/27/2016,00:00:00,REGULAR,5799442,1966041,2016-08-27
6,A002,R051,02-00-00,59 ST,NQR456,BMT,08/28/2016,00:00:00,REGULAR,5800252,1966295,2016-08-28
12,A002,R051,02-00-00,59 ST,NQR456,BMT,08/29/2016,00:00:00,REGULAR,5800934,1966546,2016-08-29
18,A002,R051,02-00-00,59 ST,NQR456,BMT,08/30/2016,00:00:00,REGULAR,5802514,1967057,2016-08-30
24,A002,R051,02-00-00,59 ST,NQR456,BMT,08/31/2016,00:00:00,REGULAR,5804071,1967579,2016-08-31


In [90]:
#Next we will combine the C/A, unit and SCP data to make 1 column for each turnstile

In [91]:
max_and_min['TURN_ID']=max_and_min['C/A']+' '+max_and_min['UNIT']+' '+max_and_min['SCP']


/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_43109/673317786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_and_min['TURN_ID']=max_and_min['C/A']+' '+max_and_min['UNIT']+' '+max_and_min['SCP']


In [92]:
#let's remove the 3 turnstile identifiers, and remove any other unnecessary columns at the moment
max_and_min.drop(columns=['LINENAME', 'DIVISION', 'DESC'], inplace=True)
max_and_min.head()

/var/folders/z4/gqr7pbzd1jx381lnrbtb0c4w0000gp/T/ipykernel_43109/2055683317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_and_min.drop(columns=['LINENAME', 'DIVISION', 'DESC'], inplace=True)


,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRIES_START,EXITS,datetime,TURN_ID
0,A002,R051,02-00-00,59 ST,08/27/2016,00:00:00,5799442,1966041,2016-08-27,A002 R051 02-00-00
6,A002,R051,02-00-00,59 ST,08/28/2016,00:00:00,5800252,1966295,2016-08-28,A002 R051 02-00-00
12,A002,R051,02-00-00,59 ST,08/29/2016,00:00:00,5800934,1966546,2016-08-29,A002 R051 02-00-00
18,A002,R051,02-00-00,59 ST,08/30/2016,00:00:00,5802514,1967057,2016-08-30,A002 R051 02-00-00
24,A002,R051,02-00-00,59 ST,08/31/2016,00:00:00,5804071,1967579,2016-08-31,A002 R051 02-00-00


In [93]:
dailydiff=(max_and_min.groupby(['TURN_ID', 'STATION', 'DATE'], as_index=False).ENTRIES_START.first())
dailydiff.head()

,TURN_ID,STATION,DATE,ENTRIES_START
0,A002 R051 02-00-00,59 ST,08/27/2016,5799442
1,A002 R051 02-00-00,59 ST,08/28/2016,5800252
2,A002 R051 02-00-00,59 ST,08/29/2016,5800934
3,A002 R051 02-00-00,59 ST,08/30/2016,5802514
4,A002 R051 02-00-00,59 ST,08/31/2016,5804071


In [94]:
#Now we need to shift the data so we know the max entries. shift back 1.
dailydiff['ENTRIES_END']=dailydiff['ENTRIES_START'].shift(-1)
dailydiff.head()

,TURN_ID,STATION,DATE,ENTRIES_START,ENTRIES_END
0,A002 R051 02-00-00,59 ST,08/27/2016,5799442,5800252.0
1,A002 R051 02-00-00,59 ST,08/28/2016,5800252,5800934.0
2,A002 R051 02-00-00,59 ST,08/29/2016,5800934,5802514.0
3,A002 R051 02-00-00,59 ST,08/30/2016,5802514,5804071.0
4,A002 R051 02-00-00,59 ST,08/31/2016,5804071,5805638.0


In [95]:
#now find the difference from end minus start
dailydiff['ENTRIES_DIFF']=dailydiff.ENTRIES_END-dailydiff.ENTRIES_START
dailydiff.head()

,TURN_ID,STATION,DATE,ENTRIES_START,ENTRIES_END,ENTRIES_DIFF
0,A002 R051 02-00-00,59 ST,08/27/2016,5799442,5800252.0,810.0
1,A002 R051 02-00-00,59 ST,08/28/2016,5800252,5800934.0,682.0
2,A002 R051 02-00-00,59 ST,08/29/2016,5800934,5802514.0,1580.0
3,A002 R051 02-00-00,59 ST,08/30/2016,5802514,5804071.0,1557.0
4,A002 R051 02-00-00,59 ST,08/31/2016,5804071,5805638.0,1567.0


In [96]:
#entries_diff is the number of new entries per turnstile per day. 

# Exercise 4

In [97]:
#For each ControlArea/UNIT/STATION combo, for each day, sum the differences
#go back to our max_and_min dataframe:
max_and_min.head()

,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRIES_START,EXITS,datetime,TURN_ID
0,A002,R051,02-00-00,59 ST,08/27/2016,00:00:00,5799442,1966041,2016-08-27,A002 R051 02-00-00
6,A002,R051,02-00-00,59 ST,08/28/2016,00:00:00,5800252,1966295,2016-08-28,A002 R051 02-00-00
12,A002,R051,02-00-00,59 ST,08/29/2016,00:00:00,5800934,1966546,2016-08-29,A002 R051 02-00-00
18,A002,R051,02-00-00,59 ST,08/30/2016,00:00:00,5802514,1967057,2016-08-30,A002 R051 02-00-00
24,A002,R051,02-00-00,59 ST,08/31/2016,00:00:00,5804071,1967579,2016-08-31,A002 R051 02-00-00


In [98]:
df_ca_u_station=max_and_min.groupby(['C/A', 'UNIT', 'STATION', 
                                     'DATE'], as_index=False).ENTRIES_START.sum()
df_ca_u_station.head()

,C/A,UNIT,STATION,DATE,ENTRIES_START
0,A002,R051,59 ST,08/27/2016,56512751
1,A002,R051,59 ST,08/28/2016,56522053
2,A002,R051,59 ST,08/29/2016,56529727
3,A002,R051,59 ST,08/30/2016,56545813
4,A002,R051,59 ST,08/31/2016,56562516


In [99]:
#make the entries end and diff columns like before
df_ca_u_station['ENTRIES_END']=df_ca_u_station['ENTRIES_START'].shift(-1)
df_ca_u_station['DAILY_ENTRIES_DIFF_BY_C/A_UNIT_STATION']=df_ca_u_station.ENTRIES_END-df_ca_u_station.ENTRIES_START
df_ca_u_station.head()


,C/A,UNIT,STATION,DATE,ENTRIES_START,ENTRIES_END,DAILY_ENTRIES_DIFF_BY_C/A_UNIT_STATION
0,A002,R051,59 ST,08/27/2016,56512751,56522053.0,9302.0
1,A002,R051,59 ST,08/28/2016,56522053,56529727.0,7674.0
2,A002,R051,59 ST,08/29/2016,56529727,56545813.0,16086.0
3,A002,R051,59 ST,08/30/2016,56545813,56562516.0,16703.0
4,A002,R051,59 ST,08/31/2016,56562516,56579136.0,16620.0


In [100]:
#DAILY_ENTRIES_DIFF_BY_C/A_UNIT_STATION tells how many new enties per 
#C/A, UNIT, STATION COMBINATION

# Exercise 5

In [101]:
#Now we need to do a daily time 
#series for each station by adding up all the turnstiles in the station
#we return to max_and_min and sort by station and date
max_and_min.head()

,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRIES_START,EXITS,datetime,TURN_ID
0,A002,R051,02-00-00,59 ST,08/27/2016,00:00:00,5799442,1966041,2016-08-27,A002 R051 02-00-00
6,A002,R051,02-00-00,59 ST,08/28/2016,00:00:00,5800252,1966295,2016-08-28,A002 R051 02-00-00
12,A002,R051,02-00-00,59 ST,08/29/2016,00:00:00,5800934,1966546,2016-08-29,A002 R051 02-00-00
18,A002,R051,02-00-00,59 ST,08/30/2016,00:00:00,5802514,1967057,2016-08-30,A002 R051 02-00-00
24,A002,R051,02-00-00,59 ST,08/31/2016,00:00:00,5804071,1967579,2016-08-31,A002 R051 02-00-00


In [116]:
ents_by_station=max_and_min.groupby(['STATION', 
                                     'DATE'], as_index=False).ENTRIES_START.sum()
ents_by_station['ENTRIES_END']=ents_by_station['ENTRIES_START'].shift(-1)
ents_by_station['DAILY_ENTRIES_DIFF_BY_STATION']=ents_by_station.ENTRIES_END-ents_by_station.ENTRIES_START
ents_by_station.head()


,STATION,DATE,ENTRIES_START,ENTRIES_END,DAILY_ENTRIES_DIFF_BY_STATION
0,1 AV,08/27/2016,1412253113,1.412269e+09,15954.0
1,1 AV,08/28/2016,1412269067,1.412282e+09,13109.0
2,1 AV,08/29/2016,1412282176,1.412301e+09,18582.0
3,1 AV,08/30/2016,1412300758,1.412320e+09,19685.0
4,1 AV,08/31/2016,1412320443,1.412340e+09,19748.0


# Question 6

In [ ]:
#Over multiple weeks, sum total ridership for each station and sort them,
#so you can find out the stations with the highest traffic during the time you investigate.

In [ ]:
#the prior question sorted entries by station by day. Using that data, we can get all the 
#entries over all the dates. 

In [113]:
df_weekly=ents_by_station.groupby(['STATION'], as_index=
                                  False).DAILY_ENTRIES_DIFF_BY_STATION.sum()
df_weekly.head()

,STATION,DAILY_ENTRIES_DIFF_BY_STATION
0,1 AV,-1.346954e+09
1,103 ST-CORONA,4.915730e+09
2,104 ST,-4.953066e+09
3,110 ST,2.969056e+07
4,111 ST,1.016732e+08


In [118]:
#These values are negative so we need to adjust df_weekly to eliminate the last date
df_weekly=ents_by_station[(ents_by_station['DATE']!='09/16/2016')].groupby(['STATION'], as_index=
                                  False).DAILY_ENTRIES_DIFF_BY_STATION.sum()
df_weekly.head()

,STATION,DAILY_ENTRIES_DIFF_BY_STATION
0,1 AV,380198.0
1,103 ST-CORONA,359940.0
2,104 ST,56714.0
3,110 ST,203541.0
4,111 ST,245677.0


In [ ]:
#finally, sort by daily_entries and rename the column

In [126]:
df_weekly.rename(columns={'DAILY_ENTRIES_DIFF_BY_STATION':'max_total_entries'}, inplace=True)
df_weekly.head()

,STATION,max_total_entries
0,1 AV,380198.0
1,103 ST-CORONA,359940.0
2,104 ST,56714.0
3,110 ST,203541.0
4,111 ST,245677.0


In [130]:
df_weekly.sort_values('max_total_entries', ascending=False, inplace=True)
df_weekly.max_total_entries=df_weekly.max_total_entries.astype('int')
df_weekly.head()

,STATION,max_total_entries
116,CHRISTOPHER ST,1145612233
55,51 ST,514821228
12,167 ST,67716759
199,TIMES SQ-42 ST,66602188
176,ORCHARD BEACH,22158253


In [ ]:
#this gives us our entries per station. There is a need for some cleansing.
#for example- Christopher Street likely did not get over 1 billion people 
#going through it in a few weeks.